In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, SGD
from torchvision import datasets

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from typing import Any

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

data_folder = '../../data/FMNIST/'
fmnist = datasets.FashionMNIST(data_folder, train=True, download=True)
tr_images = fmnist.data
tr_targets = fmnist.targets

val_fmnist = datasets.FashionMNIST(data_folder, train=False, download=True)
val_images = val_fmnist.data
val_targets = val_fmnist.targets

In [3]:
from imgaug import augmenters as iaa

Important parameters in the `Affine` method
* **scale** specifies the amount of zoom that is to be done for the image
* **translate_percent** specifies the amount of translation as a percentage of the image's height and width
* **translate_px** : specifies the amount of translation as an absolute number of pixels
* **rotate** : specifies the amount of rotation that is to be done on the image
* **shear** : specifies the amount of rotation that is to be done on part of the image
* **mode** : there are different modes we can use to fill the values of newly created pixels
    * **constant** : Pads with a constant value.
    * **edge** : Pads with the edge values of the array.
    * **symmetric** : Pads with the reflection of the vector mirrored along the edge of the array.
    * **reflect** : Pads with the reflection of the vector mirrored on the first and last values of the vector along each axis.
    * **wrap** : Pads with the wrap of the vector along the axis.

In [4]:
aug = iaa.Sequential([
    iaa.Affine(translate_px={'x':(-10,10)},
               mode='constant')
])

In [80]:
class FMNISTDataset(Dataset):
    def __init__(self, x, y, aug=None) -> None:
        super().__init__()
        self.x, self.y = x, y
        self.aug = aug
    
    def __getitem__(self, index) -> Any:
        return self.x[index], self.y[index]
    
    def __len__(self) -> int:
        return len(self.x)

    def collate_fn(self, batch):
        # logic to modify a batch of images
        # print(len(batch), batch)
        ims, classes = list(zip(*batch))
        # print(classes)
        # ims = batch
        if self.aug:
            ims1 = ims.cpu().numpy().tolist() if 'torch' in str(type(ims)) else ims
            ims = self.aug.augment_images(images=ims1)
        ims = torch.tensor(ims)[:, None, :, :].to(device)/255
        classes = torch.tensor(classes).to(device)
        return ims, classes

In [39]:
def get_data():
    train = FMNISTDataset(tr_images, tr_targets, aug=aug)
    'notice the collate_fn argument'
    trn_dl = DataLoader(train, batch_size=64,
                collate_fn=train.collate_fn, shuffle=True)
    val = FMNISTDataset(val_images, val_targets)
    val_dl = DataLoader(val, batch_size=len(val_images),
                collate_fn=val.collate_fn, shuffle=True)
    return trn_dl, val_dl

In [67]:
# train = FMNISTDataset(tr_images, tr_targets, aug=aug)
# train.collate_fn(torch.tensor(tr_images[:32]))

In [84]:
tr_images.cpu().numpy()[:10]

array([[[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        ...,
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]],

       [[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        ...,
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]],

       [[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        ...,
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]],

       ...,

       [[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0

In [40]:
def get_model():
    model = nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Conv2d(64, 128, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(3200, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
    )
    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=1e-3)
    return model, loss_fn, optimizer

In [41]:
def train_batch(x, y, model, loss_fn, optim):
    model.train()
    prediction = model(x)
    loss_val = loss_fn(prediction, y)
    loss_val.backward()
    optim.step()
    optim.zero_grad()
    return loss_val.item()

In [43]:
tr_targets

tensor([9, 0, 0,  ..., 3, 0, 5])

In [81]:
trn_dl, val_dl = get_data()
model, loss_fn, optimizer = get_model()

for epoch in range(5):
    print(epoch)
    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        print(type(x), type(y))
        batch_loss = train_batch(x, y, model, loss_fn, optimizer)

0


AttributeError: 'Tensor' object has no attribute 'deepcopy'